In [1]:
import pandas as pd
from music21 import converter, instrument, note, chord
from sklearn.preprocessing import LabelEncoder
import itertools

In [2]:
path = 'data\maestro-v3.0.0.csv'
df = pd.read_csv(path)

In [3]:
#df = df.iloc[:50]

In [4]:
#Cration du tableau contenant les musiques sous forme de note
def extract_notes(file):
    midi = converter.parse(file)
    notes = []
    for part in midi.parts:
        for element in part.flat.notes:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

def ex_notes_tab(df,col_out):
    df['midi_file_data'] = 'data/'+df['midi_filename']
    df[col_out] = df['midi_file_data'].apply(extract_notes)
    return df

In [5]:
#Encodage
def encodage(df,col):
    all_notes = []
    for k in df[col].to_list():
        all_notes+=k
    all_notes = list(set(all_notes))

    encoder = LabelEncoder()
    encoder.fit(all_notes)

    df['encoded'] = df[col].apply(encoder.transform)
    return df

In [6]:
#préparation tableau
def sequence_list(L):
    sequence_length = 500

    input_sequences = []
    output_notes = []

    for i in range(len(L) - sequence_length):
        seq_in = L[i:i + sequence_length]  
        seq_out = L[i + sequence_length]  
        input_sequences.append(seq_in)
        output_notes.append(seq_out)

    return input_sequences,output_notes

def sequence(dff):
    X,y = [],[]
    for k in dff['encoded'].to_list():
        xtemp,ytemp = sequence_list(k)
        X.append(xtemp)
        y.append(ytemp)
    X=list(itertools.chain(*X))
    y=list(itertools.chain(*y))

    return pd.DataFrame({"X":X,"y":y})
    

In [7]:
def preprocessing(df):
    print("extraction notes")
    df = ex_notes_tab(df,'notes')

    print("encodage")
    df = encodage(df,'notes')

    print("sequence train")
    train = sequence(df[df['split']=='train'])
    print("sequence test")
    test = sequence(df[df['split']=='test'])
    print("sequence val")
    val = sequence(df[df['split']=='validation'])

    return train,test,val

In [8]:
train,test,val = preprocessing(df)

extraction notes


C:\Users\ccdeb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\music21\stream\base.py:3675: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
C:\Users\ccdeb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\music21\stream\base.py:3675: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
In C:\Users\ccdeb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\matplotlib\mpl-data\stylelib\ggplot.mplstyle: .flat is deprecated.  Call .flatten() instead
C:\Users\ccdeb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\music21\stream\base.py:3675: Mu

encodage
sequence train
sequence test
sequence val


In [9]:
train.to_csv('train.csv')

In [10]:
test.to_csv('test.csv')

In [11]:
val.to_csv('val.csv')